In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from libs.rdkit_io import inchi_to_mol_file
import pandas as pd
import requests

In [2]:
df = pd.read_csv("<PATH>/State_Drug_Utilization_Data_2018.csv")
df.head()

,Utilization Type,State,Labeler Code,Product Code,Package Size,Year,Quarter,Product Name,Suppression Used,Units Reimbursed,Number of Prescriptions,Total Amount Reimbursed,Medicaid Amount Reimbursed,Non Medicaid Amount Reimbursed,Quarter Begin,Quarter Begin Date,Latitude,Longitude,Location,NDC
0,FFSU,VT,781,7243,55,2018,1,FENTANYL M,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2018 12:00:00 AM,44.0407,-72.7093,"(44.0407, -72.7093)",781724355
1,MCOU,NV,43547,222,15,2018,1,Levetirace,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2018 12:00:00 AM,38.4199,-117.1219,"(38.4199, -117.1219)",43547022215
2,FFSU,AZ,60505,3583,6,2018,1,ABACAVIR,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2018 12:00:00 AM,33.7712,-111.3877,"(33.7712, -111.3877)",60505358306
3,FFSU,NY,574,153,30,2018,1,MORPHINE S,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2018 12:00:00 AM,42.1497,-74.9384,"(42.1497, -74.9384)",574015330
4,MCOU,ND,93,73,1,2018,1,ZOLPIDEM T,True,NaN,NaN,NaN,NaN,NaN,1/1,01/01/2018 12:00:00 AM,47.5362,-99.7930,"(47.5362, -99.793)",93007301


In [3]:
#replacing all of the 10-character drug names with their full name. We are only using this ugly
#brute force method because we don't want to have to fuzzy match these names later.
df['Product Name'] = df['Product Name'].replace({
    "AMOXICILLI":"AMOXICILLIN",
    "Lisinopril":"LISINOPRIL",
    "ATAVORSTAT":"ATAVORSTATIN",
    "VENTOLIN H":"ALBUTEROL",
    "FLUTICASON":"FLUTICASONE",
    "ONDANSETRO":"ONDANSETRON",
    "PROAIR HFA":"ALBUTEROL",
    "MONTELUKAS":"MONTELUKAST",
    "LEVOTHYROX":"LEVOTHYROXINE",
    "HYDROCODON":"HYDROCODONE",
    "HYDROXYZIN":"HYDROXYZINE",
    "PROMETHIZI":"PROMETHIZINE",
    "TRAMADOL H":"TRAMADOL",
    "LOSARTAN P":"LOSARTAN",
    "AZITHROMYC":"AZITHROMYCIN",
    "CYCLOBENZA":"CYCLOBENZAPRINE",
    "ESCITALOPR":"ESCITALOPRAM",
    "PANTOPRAZO":"PANTOPRAZOLE",
    "SIMVASTATI":"SIMVASTATIN",
    "DEXTROAMPH":"DEXTROAMPHETAMINE",
    "PREDNISOLO":"PREDNISOLONE",
    "RISPERIDON":"RISPERIDONE",
    "VENLAFAXIN":"VENLAFAXINE",
    "SULFAMETHO":"SULFAMETHOXAZOLE",
    "CLINDAMYCI":"CLINDAMYCIN",
    "METRONIDAZ":"METRONIDAZOLE",
    "HYDROCORTI":"HYDROCORTISONE",
    "LAMOTRIGIN":"LAMOTRIGINE",
    "OSELTAMIVI":"OSELTAMIVIR",
    "POLYETHYLE":"POLYETHYLENE GLYCOL",
    "OXYCODONE-":"OXYCODONE",
    "Oxycodone":"OXYCODONE",
    "FLOVENT HF":"FLUTICASONE",
    "ARIPIPRAZO":"ARIPIPRAZOLE",
    "LEVETIRACE":"LEVETIRACETAM",
    "AMITRIPTYL":"AMITRIPTYLINE",
    "ACETAMINOP":"ACETAMINOPHINE",
    "ASPIRIN EC":"ASPIRIN",
    "MIRTAZAPIN":"MIRTAZAPINE",
    "ASPIR-LOW":"ASPIRIN",
    "PRAVASTATI":"PRAVASTATIN",
    "NAPROXEN 5":"NAPROXEN",
    "DOXYCYCLIN":"DOXYCYCLINE",
    "SUBOXONE 8":"SUBOXONE",
    "Suboxone S":"SUBOXONE",
    "METHYLPRED":"METHYLPREDNISOLONE",
    "CIPROFLOXA":"CIPROFLOXACIN",
    "TAMIFLU 6":"OSELTAMIVIR",
    "BUPRENORPH":"BUPRENORPHINE",
    "CLOPIDOGRE":"CLOPIDOGREL",
    "NITROFURAN":"NITROFURANTOIN",
    "ZOLPIDEM T":"ZOLPIDEM",
    "CLOTRIMAZO":"CLOTRIMAZOLE",
    "SUMATRIPTA":"SUMATRIPTAN",
    "DIPHENHYDR":"DIPHENHYDRAMINE",
    "OXCARBAZEP":"OXCARBAZEPINE",
    "DEXAMETHAS":"DEXAMETHASONE",
    "NYSTATIN 1":"NYSTATIN",
    "Ibuprofen":"IBUPROFEN",
    "METHOCARBA":"METHOCARBAMOL",
    "MEDROXYPRO":"MEDROXYPROGESTERONE",
    "VALACYCLOV":"VALACYCLOVIR",
    "IPRATROPIU":"IPRATROPIUM",
    "DICYCLOMIN":"DICYCLOMINE",
    "LITHIUM CA":"LITHIUM",
    "LOSARTAN-H":"LOSARTAN",
    "FENOFIBRAT":"FENOFIBRATE",
    "BACLOFEN 1":"BACLOFEN",
    "DOCUSATE S":"DOCUSATE",
    "CYANOCOBAL":"CYANOCOBALAMIN",
    "GLIMEPIRID":"GLIMEPIRIDE",
    "ALLOPURINO":"ALLOPURINOL",
    "CEFTRIAXON":"CEFTRIAXONE",
    "MIXED AMPH":"ADDERALL",
    "ADDERALL X":"ADDERALL",
    "BROMPHENIR":"BROMPHENIRAMINE",
    "BENZONATAT":"BENZONATATE",
    "PROPRANOLO":"PROPRANOLOL",
    "FOCALIN XR":"FOCALIN",
    "ERYTHROMYC":"ERYTHROMYCIN",
    "KETOCONAZO":"KETOCONAZOLE",
    "HUMALOG 10":"HUMALOG",
    "MORPHINE S":"MORPHINE",
    "CEFDINIR 2":"CEFDINIR",
    "SODIUM CHL":"SODIUM CHLORIDE",
    "BASAGLAR 1":"BASAGLAR",
    "HALOPERIDO":"HALOPERIDOL",
    "NOVOLOG 10":"NOVOLOG",
    "Omeprazole":"OMEPRAZOLE",
    "ALENDRONAT":"ALENDRONATE",
    "WARFARIN S":"WARFARIN",
    "LANTUS 3ML":"LANTUS",
    "Montelukas":"MONTELUKAST",
    "Gabapentin":"GABAPENTIN",
    "LATANOPROS":"LATANOPROST",
    "HYDRALAZIN":"HYDRALAZINE",
    "DEXMETHYLP":"DEXMETHYLPHENIDATE",
    "METOCLOPRA":"METOCLOPRAMIDE",
    "CHLORHEXID":"CHLORHEXIDINE",
    "ATOMOXETIN":"ATOMOXETIN",
    "LEVOFLOXAC":"LEVOFLOXACIN",
    "ASPIRIN LO":"ASPIRIN",
    "Ergocalcif":"ERGOCALCIFEROL",
    "TRIMETHOPR":"TRIMETHOPRIM",
    "ASPIRIN 81":"ASPIRIN",
    "CARBAMAZEP":"CARBAMAZEPINE",
    "CYPROHEPTA":"CYPROHEPTADINE",
    "JANUVIA 10":"JANUVIA",
    "NORETHINDR":"NORETHINDRONE",
    "Metformin":"METFORMIN",
    "ZOLPIDEM 1":"ZOLPIDEM",
    "MAPAP 500":"MAPAP",
    "Quetiapine":"QUETIAPINE",
    "VYVANSE 30":"VYVANSE",
    "DOK 100 MG":"DOCUSATE",
    "ROSUVASTAT":"ROSUVASTATIN",
    "DEPO-MEDRO":"METHYLPREDNISOLONE",
    "KENALOG-40":"TRIAMCINOLONE",
    "ZIPRASIDON":"ZIPRASIDONE",
    "Ondansetro":"ONDANSETRON",
    "NORTRIPTYL":"NORTRIPTYLINE",
    "GEMFIBROZI":"GEMFIBROZIL",
    "OXYCODONE/":"OXYCODONE",
    "CHLORTHALI":"CHLORTHALIDONE",
    "METHOTREXA":"METHOTREXATE",
    "HYDROXYCHL":"HYDROXYCHLOROQUINE",
    "DIAZEPAM 5":"DIAZEPAM",
    "BREO ELLIP":"FLUTICASONE",
    "VYVANSE 40":"VYVANSE",
    "AMOX CLAV": "CLAVULANIC ACID",
    "BETAMETHAS":"BETAMETHASONE",
    "LANTUS 100":"LANTUS",
    "EPINEPHRIN":"EPINEPHRINE",
    "PIOGLITAZO":"PIOGLITAZONE",
    "HECTOROL 4":"HECTOROL",
    "EPOGEN 20,":"EPOETIN",
    "NEOMYCIN-P":"NEOMYCIN",
    "SPIRIVA 30":"SPIRIVA",
    "HYDROCHLOR":"HYDROCHLOROTHIAZIDE",
    "Hydrochlor":"HYDROCHLOROTHIAZIDE",
    "BENZOYL PE":"BENZOYL PEROXIDE"
    
})

In [4]:
#processing the resultant dataframe after replacement.

#group by product name, then sum the number of prescriptions of the drug
products_by_prescrip = df.groupby("Product Name").sum()["Number of Prescriptions"]

#sort the drugs by descending number of prescriptions, and only select the top 200
top200 = products_by_prescrip.sort_values(0,ascending=False)[0:200]

#dataframe processing to select only the Product Names
top200 = pd.DataFrame(top200)
top200 = top200.reset_index()
top200_names = top200["Product Name"].unique()

In [5]:
top200_names

array(['ALBUTEROL', 'AMOXICILLIN', 'IBUPROFEN', 'LISINOPRIL',
       'FLUTICASONE', 'GABAPENTIN', 'ATORVASTAT', 'METFORMIN',
       'OMEPRAZOLE', 'CETIRIZINE', 'LORATADINE', 'HYDROCODONE',
       'AMLODIPINE', 'LEVOTHYROXINE', 'MONTELUKAST', 'ONDANSETRON',
       'SERTRALINE', 'AZITHROMYCIN', 'OXYCODONE', 'TRAZODONE',
       'METOPROLOL', 'ASPIRIN', 'PREDNISONE', 'VITAMIN D2', 'FLUOXETINE',
       'QUETIAPINE', 'RANITIDINE', 'LOSARTAN', 'BUPROPION', 'CLONIDINE',
       'HYDROXYZINE', 'HYDROCHLOROTHIAZIDE', 'CYCLOBENZAPRINE',
       'METHYLPHEN', 'ALPRAZOLAM', 'SIMVASTATIN', 'CLONAZEPAM',
       'ESCITALOPRAM', 'PANTOPRAZOLE', 'PROMETHAZI', 'TRIAMCINOL',
       'CITALOPRAM', 'CEPHALEXIN', 'DEXTROAMPHETAMINE', 'FUROSEMIDE',
       'OSELTAMIVIR', 'PREDNISOLONE', 'DULOXETINE', 'RISPERIDONE',
       'FOLIC ACID', 'GUANFACINE', 'TRAMADOL', 'VENLAFAXINE', 'SUBOXONE',
       'SULFAMETHOXAZOLE', 'CLINDAMYCIN', 'BUSPIRONE', 'TOPIRAMATE',
       'FLUCONAZOL', 'DIVALPROEX', 'LAMOTRIGINE', 'HYDROCO

In [ ]:
#create an empty list to hold each of the "failure" request drugs
search_failure = []
#loop over each of the top 200 names, converting the name into an InChI using the chemical resolver API
for drug in top200_names:
    res = requests.get(f"https://cactus.nci.nih.gov/chemical/structure/{drug}/stdinchi")
    
    if res.status_code == 404:
        search_failure.append(drug)
    else:
        inchi_to_mol_file(res.text, drug)